In [1]:
!pip install transformers datasets accelerate


In [2]:
from datasets import load_from_disk

# Define the dataset path
dataset_path = "/kaggle/input/preprocessed-dataset/preprocessed_dataset"

# Load the dataset
dataset = load_from_disk(dataset_path)

# Inspect the dataset
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['wav', 'input_features', 'labels'],
        num_rows: 6827
    })
    validation: Dataset({
        features: ['wav', 'input_features', 'labels'],
        num_rows: 2252
    })
    test: Dataset({
        features: ['wav', 'input_features', 'labels'],
        num_rows: 2267
    })
})


In [3]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load Whisper processor and model
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-darija",
    per_device_train_batch_size=4,  # Adjust batch size if needed
    gradient_accumulation_steps=4,  # Effective batch size = 4 * 4 = 16
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    num_train_epochs=3,
    learning_rate=1e-5,
    warmup_steps=500,
    save_total_limit=2,
    fp16=True,  # Mixed precision for GPU
    predict_with_generate=True,
    report_to="none",
)


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
def data_collator(features):
    input_features = [feature["input_features"] for feature in features]
    labels = [feature["labels"] for feature in features]

    # Pad input features and labels
    batch = processor.feature_extractor.pad({"input_features": input_features}, return_tensors="pt")
    labels_batch = processor.tokenizer.pad({"input_ids": labels}, return_tensors="pt", padding=True)

    # Replace padding token id with -100 to ignore during loss computation
    labels_batch["input_ids"][labels_batch["input_ids"] == processor.tokenizer.pad_token_id] = -100

    # Combine into a single batch
    batch["labels"] = labels_batch["input_ids"]
    return batch


In [7]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
)
trainer.train()


<ipython-input-7-5585130f1586>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,1.555100,1.464974
1000,0.313500,0.449092


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1278, training_loss=1.6680029449701683, metrics={'train_runtime': 8099.0809, 'train_samples_per_second': 2.529, 'train_steps_per_second': 0.158, 'total_flos': 5.89810841174016e+18, 'train_loss': 1.6680029449701683, 'epoch': 2.9935559461042764})

In [9]:
# Save the fine-tuned model
model.save_pretrained("/kaggle/working/whisper-darija")
processor.save_pretrained("/kaggle/working/whisper-darija")

print("Model saved to /kaggle/working/whisper-darija")


Model saved to /kaggle/working/whisper-darija


In [10]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [13]:
!pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.0 MB/s eta 0:00:00a 0:00:01


In [15]:
# Add the language to the processor during generation
with torch.no_grad():
    predicted_ids = model.generate(
        inputs.input_features,
        forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
    )



NameError: name 'torch' is not defined

In [ ]:
import torch
import evaluate

# Load the WER metric
wer_metric = evaluate.load("wer")

# Generate predictions with specified language and attention mask
predictions = trainer.predict(dataset["test"])

# Decode predictions and references
decoded_preds = processor.batch_decode(predictions.predictions, skip_special_tokens=True)
decoded_refs = [processor.tokenizer.decode(item["labels"], skip_special_tokens=True) for item in dataset["test"]]

# Compute Word Error Rate
wer = wer_metric.compute(predictions=decoded_preds, references=decoded_refs)
print(f"Word Error Rate (WER): {wer}")


In [16]:
pip install sounddevice scipy transformers


Note: you may need to restart the kernel to use updated packages.


In [20]:
!pip install pipwin
!pipwin install pyaudio


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:0000:01
  Created wheel for pipwin: filename=pipwin-0.5.2-py2.py3-none-any.whl size=8766 sha256=ab8af29ae67ae606ad570574d20e017e28a25b9cb23c81adb24ef99bf369ab51
  Stored in directory: /root/.cache/pip/wheels/b4/2c/53/c5a91c548b9f030b592608c24efda23ff966b1cceac6414765
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=88d0b55f09f7113a5bb0fee5cb35293610c47c62d5810706868545bf4ff05258
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=25983 sha256=ac7a8184a0dbab061b0be255868bd4c856a3e0a037f7fde95893e23dec320207
  Stored in directory: /root/.cache/pip/wheels/5e/81/26/5956478df303e2bf5a85a5df595bb307bd25948a4bab69f7c7
Successfu

In [25]:
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

# Load the fine-tuned model
model_path = "./whisper-darija"
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path)

# Path to the audio file
audio_path = "/kaggle/input/audioali/mic_audio2.wav"

# Load the audio file
waveform, sample_rate = torchaudio.load(audio_path)

# Resample the audio to 16 kHz (required by Whisper)
if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)

# Process the audio input
inputs = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt")

# Transcribe the audio
with torch.no_grad():
    predicted_ids = model.generate(
        inputs.input_features,
        forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
    )

# Decode the transcription
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print(f"Transcription: {transcription}")


Transcription: معليكم بخير لبس عليكم شكرا عودوه تمم زينين الحياه كدهيه


In [27]:
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch

# Load the fine-tuned model
model_path = "./whisper-darija"
processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path)

# Path to the audio file
audio_path = "/kaggle/input/audioali/mic_audio4.wav"

# Load the audio file
waveform, sample_rate = torchaudio.load(audio_path)

# Resample the audio to 16 kHz (required by Whisper)
if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)

# Process the audio input
inputs = processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt")

# Transcribe the audio
with torch.no_grad():
    predicted_ids = model.generate(
        inputs.input_features,
        forced_decoder_ids=processor.get_decoder_prompt_ids(language="ar", task="transcribe")
    )

# Decode the transcription
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print(f"Transcription: {transcription}")


Transcription: انا علي اصواب انا وصاحبي احمد ابن براهي ومحمد عيسان ترنهت بروج بس يبينو بلي يمكن نفاين شونيه ويسبر علي الدريجه بشوي يقدر يفهم اللغه ديناه زوينه وعليش لانك نشو بابيده
